In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from bs4 import BeautifulSoup
import requests
import re

In [2]:
url_1_1 = "http://www.chakoteya.net/DoctorWho/1-1.htm"
page_response_1_1 = requests.get(url_1_1, timeout = 5)
page_content_1_1 = BeautifulSoup(page_response_1_1.content, "html.parser")

In [3]:
page_content_1_1

<!DOCTYPE html>

<html lang="en-gb">
<head>
<meta charset="utf-8"/>
<meta content="Doctor Who, Episodes, Transcripts, An Unearthly Child, The Forest of Fear, The Cave of Skulls, The Firemaker, Doctor Who, Ian Chesterton, Barbara Wright, Susan Foreman, Za, Hur, Kal, Horg, Mother, William Hartnell" http-equiv="keywords"/>
<meta content="Doctor Who, Transcripts, Unearthly Child" name="Keywords"/>
<meta content="Doctor Who episode transcripts" name="Description"/>
<title>The Doctor Who Transcripts - An Unearthly Child</title>
</head>
<body bgcolor="#d5d5d5">
<font color="#2867d0" face="Arial, Helvetica, sans-serif" size="5"><b>An Unearthly Child</b></font><br/>
<p> <font face="Arial, Helvetica, sans-serif" size="2">
Original Airdate: 23 Nov, 1963<br/>
</font></p>
<div align="center">
<center>
<table border="0" width="85%">
<tbody>
<tr>
<td align="left" width="85%"> <font face="Arial, Helvetica, sans-serif" size="2"> (A
foggy night, and a policeman is patrolling his beat past I M Foreman's


In [4]:
textContent = []
paragraphs = page_content_1_1.find_all("p")
paragraphs
for i in range(len(paragraphs)):
    textContent.append(re.sub(r"[\r\n]+", r"", paragraphs[i].text))
textContent[8]

" (Ian and Barbara are parked up.) BARBARA: Over there. IAN: We're lucky there was no fog. I'd never have found this. BARBARA: Well, she doesn't seem to have arrived yet. I suppose we aredoing the right thing, aren't we? IAN: You can't justify curiosity. BARBARA: But her homework? IAN: A bit of an excuse, really, isn't it? I've seen far worse. Thetruth is, we're both curious about Susan and we won't be happy until weknow some of the answers. BARBARA: You can't just pass it off like that. If I thought I was justbeing a busybody, I'd go straight home. I thought you agreed she was abit of a mystery. IAN: Yes, but I think you'll find there's a very simple explanation toall this. BARBARA: Well, I don't know how you explain the fact that a fifteenyear old girl does not know how many shillings there are in a pound. IAN: Really? BARBARA: Really. She said she thought we were on the decimal system. IAN: Decimal system? "

In [5]:
list(set(re.findall(r"[A-Z]+:", textContent[8])))

['IAN:', 'BARBARA:']

In [6]:
def drop_colon(string):
    return string[:(len(string) - 1)]

In [7]:
elem_matches = []
for elem in textContent:
    matches = list(set(map(drop_colon, re.findall(r"[A-Z]+:", elem))))
    if len(matches) > 0:
        elem_matches.append(matches)

In [8]:
elem_matches

[['BARBARA', 'GIRL', 'BOY'],
 ['BARBARA', 'IAN'],
 ['BARBARA', 'SUSAN', 'IAN'],
 ['BARBARA', 'IAN'],
 ['BARBARA', 'SUSAN'],
 ['BARBARA', 'IAN'],
 ['SUSAN', 'IAN'],
 ['BARBARA', 'IAN'],
 ['SUSAN', 'IAN'],
 ['BARBARA', 'IAN'],
 ['BARBARA', 'IAN', 'DOCTOR'],
 ['BARBARA', 'DOCTOR', 'IAN', 'SUSAN'],
 ['HUR', 'MOTHER', 'ZA'],
 ['BARBARA', 'IAN', 'DOCTOR', 'SUSAN'],
 ['BARBARA', 'IAN', 'DOCTOR', 'SUSAN'],
 ['MOTHER', 'HUR', 'KAL', 'SUSAN', 'ALL', 'HORG', 'DOCTOR', 'ZA'],
 ['BARBARA', 'IAN', 'DOCTOR', 'SUSAN'],
 ['BARBARA', 'MOTHER', 'IAN', 'SUSAN', 'DOCTOR'],
 ['HUR', 'ZA'],
 ['DOCTOR', 'MOTHER'],
 ['HUR', 'ZA'],
 ['HUR', 'IAN', 'MOTHER', 'ZA'],
 ['BARBARA', 'HUR', 'IAN', 'SUSAN', 'DOCTOR', 'ZA'],
 ['KAL', 'MOTHER'],
 ['BARBARA', 'HUR', 'IAN', 'SUSAN', 'DOCTOR', 'ZA'],
 ['KAL', 'HORG'],
 ['KAL', 'HORG'],
 ['IAN', 'SUSAN'],
 ['IAN'],
 ['HUR', 'KAL', 'IAN', 'TRIBE', 'HORG', 'DOCTOR', 'ZA'],
 ['DOCTOR'],
 ['HUR', 'ZA'],
 ['BARBARA', 'SUSAN', 'IAN'],
 ['ZA'],
 ['BARBARA', 'IAN', 'SUSAN', 'DOCTOR'

In [129]:
class InteractionNode:
    def __init__(self, name, id_num):
        self.name = name
        self.adjacents = {}
        self.id = id_num
        
    def addAdj(self, adj):
        if adj in self.adjacents.keys():
            self.adjacents[adj] += 1
        else:
            self.adjacents[adj] = 1      

class InteractionGraph:
    def __init__(self):
        self.nodes = set()
        self.node_names = {}
        
    def addNode(self, node):
        self.nodes.add(node)
        self.node_names[node.name] = node
        
    def removeNode(self, node):
        self.nodes.remove(node)
        del self.node_names[node.name]

In [131]:
# let's experiment with one episode
ep_1_1 = InteractionGraph()
id_num = 0
for interaction in elem_matches:
    for char in interaction:
        if char not in ep_1_1.node_names:
            # edit the existing node
            new_node = InteractionNode(char, id_num)
            id_num += 1
            ep_1_1.addNode(new_node)
        old_node = ep_1_1.node_names[char]
        spec_interaction = interaction[:]
        spec_interaction.remove(char)
        if spec_interaction != None:
            for adj in spec_interaction:
                if adj in ep_1_1.node_names.keys():
                    old_node.addAdj(adj)
                else:
                    new_adj = InteractionNode(adj, id_num)
                    id_num += 1
                    ep_1_1.addNode(new_adj)
                    old_node.addAdj(adj)
        else:
            if ep_1_1.node_names[char] == None:
                # create a new node
                new_node = InteractionNode(char, id_num)
                id_num += 1
                # add the node to the interaction graph
                ep_1_1.addNode(new_node)
ep_1_1.node_names

{'ACOMAT': <__main__.InteractionNode at 0x1a1fb776d8>,
 'ADMINISTRATOR': <__main__.InteractionNode at 0x1a1d88cd30>,
 'ALL': <__main__.InteractionNode at 0x1a1d882780>,
 'ALTOS': <__main__.InteractionNode at 0x1a1fb77710>,
 'ALYDON': <__main__.InteractionNode at 0x1a1d8822e8>,
 'ANTODUS': <__main__.InteractionNode at 0x1a1fb77d30>,
 'ARBITAN': <__main__.InteractionNode at 0x1a1fb77278>,
 'AUTLOC': <__main__.InteractionNode at 0x1a1d88cba8>,
 'AYDEN': <__main__.InteractionNode at 0x1a1d8967f0>,
 'AYSEN': <__main__.InteractionNode at 0x1a1d8964a8>,
 'BARBARA': <__main__.InteractionNode at 0x1a1fb72668>,
 'BOTH': <__main__.InteractionNode at 0x1a1d88ccf8>,
 'BOY': <__main__.InteractionNode at 0x1a1d882ba8>,
 'CAMECA': <__main__.InteractionNode at 0x1a1d88cb70>,
 'CAPTAIN': <__main__.InteractionNode at 0x1a1d896588>,
 'CAROL': <__main__.InteractionNode at 0x1a1d88c748>,
 'CHENCHU': <__main__.InteractionNode at 0x1a1fb77588>,
 'CHO': <__main__.InteractionNode at 0x1a1fb77550>,
 'CLERK': <__

In [133]:
print(list(ep_1_1.node_names['BARBARA'].adjacents))
ep_1_1.node_names['BARBARA'].adjacents

['GIRL', 'BOY', 'IAN', 'SUSAN', 'DOCTOR', 'MOTHER', 'HUR', 'ZA', 'DALEK', 'ALYDON', 'ELYON', 'GANATUS', 'ALL', 'DYONI', 'ANTODUS', 'KRISTAS', 'POLO', 'TEGANA', 'CHO', 'TAU', 'LO', 'KHAN', 'VIZIER', 'ARBITAN', 'ALTOS', 'SABETHA', 'MORPHO', 'DARRIUS', 'VASAR', 'CLERK', 'TARRON', 'AYDEN', 'KALA', 'EYESEN', 'AYSEN', 'JUDGE', 'AUTLOC', 'TLOTOXL', 'MAN', 'CAPTAIN', 'VICTIM', 'IXTA', 'TONILA', 'CAROL', 'MAITLAND', 'JOHN', 'ELDER', 'ADMINISTRATOR', 'BOTH', 'COMMANDER']


{'ADMINISTRATOR': 1,
 'ALL': 3,
 'ALTOS': 21,
 'ALYDON': 6,
 'ANTODUS': 5,
 'ARBITAN': 3,
 'AUTLOC': 12,
 'AYDEN': 1,
 'AYSEN': 1,
 'BOTH': 1,
 'BOY': 1,
 'CAPTAIN': 1,
 'CAROL': 4,
 'CHO': 10,
 'CLERK': 3,
 'COMMANDER': 1,
 'DALEK': 2,
 'DARRIUS': 2,
 'DOCTOR': 102,
 'DYONI': 3,
 'ELDER': 4,
 'ELYON': 2,
 'EYESEN': 2,
 'GANATUS': 12,
 'GIRL': 1,
 'HUR': 3,
 'IAN': 149,
 'IXTA': 1,
 'JOHN': 9,
 'JUDGE': 1,
 'KALA': 3,
 'KHAN': 1,
 'KRISTAS': 6,
 'LO': 1,
 'MAITLAND': 4,
 'MAN': 1,
 'MORPHO': 1,
 'MOTHER': 1,
 'POLO': 27,
 'SABETHA': 15,
 'SUSAN': 112,
 'TARRON': 3,
 'TAU': 3,
 'TEGANA': 15,
 'TLOTOXL': 12,
 'TONILA': 3,
 'VASAR': 7,
 'VICTIM': 2,
 'VIZIER': 1,
 'ZA': 4}

In [166]:
# generalize into utility functions
def access_url(doctor, episode):
    if doctor == 37:
        url = "http://www.chakoteya.net/DoctorWho/" + str(doctor) + "-" + str(episode) + ".html"
    else:
        url = "http://www.chakoteya.net/DoctorWho/" + str(doctor) + "-" + str(episode) + ".htm"
    print(url)
    page_response = requests.get(url, timeout = 50)
    return page_response

def get_content(page_response):
    page_content = BeautifulSoup(page_response.content, "html.parser")
    textContent = []
    paragraphs = page_content.find_all("p")
    for i in range(len(paragraphs)):
        textContent.append(re.sub(r"[\r\n]+", r"", paragraphs[i].text))
    return textContent
    
def sep_chars(textContent):
    elem_matches = []
    for elem in textContent:
        if type(elem) != str:
            continue
        matches = list(set(map(drop_colon, re.findall(r"[A-Z]+:", elem))))
        if len(matches) > 0:
            elem_matches.append(matches)
    return elem_matches
    
def gen_interaction_graph(elem_matches):
    ep = InteractionGraph()
    id_num = 0
    for interaction in elem_matches:
        for char in interaction:
            if char not in ep.node_names:
                # edit the existing node
                new_node = InteractionNode(char, id_num)
                id_num += 1
                ep.addNode(new_node)
            old_node = ep.node_names[char]
            spec_interaction = interaction[:]
            spec_interaction.remove(char)
            if spec_interaction != None:
                for adj in spec_interaction:
                    if adj in ep.node_names.keys():
                        old_node.addAdj(adj)
                    else:
                        new_adj = InteractionNode(adj, id_num)
                        id_num += 1
                        ep.addNode(new_adj)
                        old_node.addAdj(adj)
            else:
                if ep.node_names[char] == None:
                    # create a new node
                    new_node = InteractionNode(char, id_num)
                    id_num += 1
                    # add the node to the interaction graph
                    ep.addNode(new_node)
    return ep

In [98]:
import time

In [137]:
# repeat for all episodes for 1st Doctor
ep_range = range(1, 8)
doctor = 1
page_content = []
for ep in ep_range:
    page_response = access_url(doctor, ep)
    page_content.extend(get_content(page_response))
    print(len(page_content))
    time.sleep(5)
elem_matches = sep_chars(page_content)
ep_graph = gen_interaction_graph(elem_matches)

http://www.chakoteya.net/DoctorWho/1-1.htm
93
http://www.chakoteya.net/DoctorWho/1-2.htm
272
http://www.chakoteya.net/DoctorWho/1-3.htm
306
http://www.chakoteya.net/DoctorWho/1-4.htm
507
http://www.chakoteya.net/DoctorWho/1-5.htm
685
http://www.chakoteya.net/DoctorWho/1-6.htm
835
http://www.chakoteya.net/DoctorWho/1-7.htm
1024


In [138]:
ep_graph.node_names

{'ACOMAT': <__main__.InteractionNode at 0x1a1e9e2cf8>,
 'ADMINISTRATOR': <__main__.InteractionNode at 0x1a1ea04668>,
 'ALL': <__main__.InteractionNode at 0x1a1e9e2b38>,
 'ALTOS': <__main__.InteractionNode at 0x1a1e9e2470>,
 'ALYDON': <__main__.InteractionNode at 0x1a1e9e2160>,
 'ANTODUS': <__main__.InteractionNode at 0x1a1e9e2ba8>,
 'ARBITAN': <__main__.InteractionNode at 0x1a1e9e2a20>,
 'AUTLOC': <__main__.InteractionNode at 0x1a1ea040f0>,
 'AYDEN': <__main__.InteractionNode at 0x1a1e9e2278>,
 'AYSEN': <__main__.InteractionNode at 0x1a1e9e2ef0>,
 'BARBARA': <__main__.InteractionNode at 0x1a1ea08b70>,
 'BOTH': <__main__.InteractionNode at 0x1a1ea040b8>,
 'BOY': <__main__.InteractionNode at 0x1a1e9e27b8>,
 'CAMECA': <__main__.InteractionNode at 0x1a1ea04208>,
 'CAPTAIN': <__main__.InteractionNode at 0x1a1ea04358>,
 'CAROL': <__main__.InteractionNode at 0x1a1ea04128>,
 'CHENCHU': <__main__.InteractionNode at 0x1a1e9e2da0>,
 'CHO': <__main__.InteractionNode at 0x1a1e9e2630>,
 'CLERK': <__

In [139]:
page_content[6]
list(set(map(drop_colon, re.findall(r"[A-Z]+:", page_content[6]))))

['BARBARA', 'SUSAN', 'IAN']

In [140]:
ep_graph.node_names['BARBARA'].adjacents

{'ADMINISTRATOR': 1,
 'ALL': 3,
 'ALTOS': 21,
 'ALYDON': 6,
 'ANTODUS': 5,
 'ARBITAN': 3,
 'AUTLOC': 12,
 'AYDEN': 1,
 'AYSEN': 1,
 'BOTH': 1,
 'BOY': 1,
 'CAPTAIN': 1,
 'CAROL': 4,
 'CHO': 10,
 'CLERK': 3,
 'COMMANDER': 1,
 'DALEK': 2,
 'DARRIUS': 2,
 'DOCTOR': 102,
 'DYONI': 3,
 'ELDER': 4,
 'ELYON': 2,
 'EYESEN': 2,
 'GANATUS': 12,
 'GIRL': 1,
 'HUR': 3,
 'IAN': 149,
 'IXTA': 1,
 'JOHN': 9,
 'JUDGE': 1,
 'KALA': 3,
 'KHAN': 1,
 'KRISTAS': 6,
 'LO': 1,
 'MAITLAND': 4,
 'MAN': 1,
 'MORPHO': 1,
 'MOTHER': 1,
 'POLO': 27,
 'SABETHA': 15,
 'SUSAN': 112,
 'TARRON': 3,
 'TAU': 3,
 'TEGANA': 15,
 'TLOTOXL': 12,
 'TONILA': 3,
 'VASAR': 7,
 'VICTIM': 2,
 'VIZIER': 1,
 'ZA': 4}

In [107]:
ep_graph.node_names['DOCTOR'].adjacents

{'ACOMAT': 1,
 'ADMINISTRATOR': 2,
 'ALL': 4,
 'ALTOS': 9,
 'ALYDON': 8,
 'ANTODUS': 1,
 'ARBITAN': 3,
 'AUTLOC': 9,
 'AYSEN': 1,
 'BARBARA': 102,
 'CAMECA': 8,
 'CAPTAIN': 1,
 'CAROL': 13,
 'CHENCHU': 1,
 'CHO': 13,
 'CLERK': 2,
 'COMMANDER': 2,
 'DALEK': 4,
 'DYONI': 3,
 'ELDER': 8,
 'ELYON': 1,
 'EMPRESS': 1,
 'ENGINEER': 1,
 'EYESEN': 3,
 'GANATUS': 4,
 'HORG': 2,
 'HUR': 5,
 'IAN': 141,
 'IXTA': 4,
 'JOHN': 5,
 'JUDGE': 2,
 'KAL': 2,
 'KHAN': 3,
 'LO': 1,
 'MAITLAND': 10,
 'MOTHER': 3,
 'POLO': 21,
 'SABETHA': 6,
 'SCIENTIST': 4,
 'SUSAN': 122,
 'TARRON': 3,
 'TAU': 4,
 'TEGANA': 11,
 'THAL': 1,
 'TLOTOXL': 9,
 'TONILA': 1,
 'TRIBE': 1,
 'VICTIM': 1,
 'VIZIER': 1,
 'WARRIOR': 4,
 'ZA': 7}

In [141]:
# let's generalize this to every doctor in every episode
def doctor_graph(ep_range, season_range):
    page_content = []
    for season in season_range:
        for ep in ep_range[str(season)]:
            try:
                print(ep)
                page_response = access_url(season, ep)
                page_content.extend(get_content(page_response))
                print(len(page_content))
                time.sleep(5)
            except:
                print("except")
                continue
    elem_matches = sep_chars(page_content)
    ep_graph = gen_interaction_graph(elem_matches)
    return ep_graph

In [142]:
doctor_1 = doctor_graph({"1": range(1, 10), "2": range(1, 10), "3": range(1, 10), "4": range(1, 3)}, range(1, 5))

1
http://www.chakoteya.net/DoctorWho/1-1.htm
93
2
http://www.chakoteya.net/DoctorWho/1-2.htm
272
3
http://www.chakoteya.net/DoctorWho/1-3.htm
306
4
http://www.chakoteya.net/DoctorWho/1-4.htm
507
5
http://www.chakoteya.net/DoctorWho/1-5.htm
685
6
http://www.chakoteya.net/DoctorWho/1-6.htm
835
7
http://www.chakoteya.net/DoctorWho/1-7.htm
1024
8
http://www.chakoteya.net/DoctorWho/1-8.htm
1227
9
http://www.chakoteya.net/DoctorWho/1-9.htm
1227
1
http://www.chakoteya.net/DoctorWho/2-1.htm
1399
2
http://www.chakoteya.net/DoctorWho/2-2.htm
1635
3
http://www.chakoteya.net/DoctorWho/2-3.htm
1726
4
http://www.chakoteya.net/DoctorWho/2-4.htm
1888
5
http://www.chakoteya.net/DoctorWho/2-5.htm
2080
6
http://www.chakoteya.net/DoctorWho/2-6.htm
2187
7
http://www.chakoteya.net/DoctorWho/2-7.htm
2343
8
http://www.chakoteya.net/DoctorWho/2-8.htm
2589
9
http://www.chakoteya.net/DoctorWho/2-9.htm
2749
1
http://www.chakoteya.net/DoctorWho/3-1.htm
2866
2
http://www.chakoteya.net/DoctorWho/3-2.htm
2976
3
http:

In [114]:
[i for i in range(1, 5)]

[1, 2, 3, 4]

In [115]:
ep_range = {1: range(1, 10), 2: range(1, 10), 3: range(1, 10), 4: range(1, 3)}
ep_range[1]

range(1, 10)

In [146]:
doctor_1.node_names["VICTIM"].id

57

In [147]:
nodes_doctor_1 = []
for char_name, char_node in doctor_1.node_names.items():
    nodes_doctor_1.append({
        "character": char_name,
        "id": char_node.id,
        "influence": len(char_node.adjacents),
    })

In [148]:
nodes_doctor_1

[{'character': 'BARBARA', 'id': 0, 'influence': 130},
 {'character': 'GIRL', 'id': 1, 'influence': 22},
 {'character': 'BOY', 'id': 2, 'influence': 2},
 {'character': 'IAN', 'id': 3, 'influence': 130},
 {'character': 'SUSAN', 'id': 4, 'influence': 78},
 {'character': 'DOCTOR', 'id': 5, 'influence': 225},
 {'character': 'HUR', 'id': 6, 'influence': 10},
 {'character': 'MOTHER', 'id': 7, 'influence': 9},
 {'character': 'ZA', 'id': 8, 'influence': 10},
 {'character': 'KAL', 'id': 9, 'influence': 9},
 {'character': 'ALL', 'id': 10, 'influence': 53},
 {'character': 'HORG', 'id': 11, 'influence': 9},
 {'character': 'TRIBE', 'id': 12, 'influence': 6},
 {'character': 'DALEK', 'id': 13, 'influence': 41},
 {'character': 'ALYDON', 'id': 14, 'influence': 14},
 {'character': 'MAN', 'id': 15, 'influence': 34},
 {'character': 'TEMMOSUS', 'id': 16, 'influence': 5},
 {'character': 'GANATUS', 'id': 17, 'influence': 13},
 {'character': 'DYONI', 'id': 18, 'influence': 9},
 {'character': 'ELYON', 'id': 19,

In [152]:
links_doctor_1 = []
for char_name, char_node in doctor_1.node_names.items():
    char_adj = list(char_node.adjacents)
    for adj in char_adj:
        target_node = doctor_1.node_names[adj]
        links_doctor_1.append({
            "source": char_node.id,
            "target": target_node.id,
            "weight": char_node.adjacents[target_node.name],
        })

In [153]:
links_doctor_1

[{'source': 0, 'target': 1, 'weight': 3},
 {'source': 0, 'target': 2, 'weight': 1},
 {'source': 0, 'target': 3, 'weight': 273},
 {'source': 0, 'target': 4, 'weight': 156},
 {'source': 0, 'target': 5, 'weight': 212},
 {'source': 0, 'target': 7, 'weight': 1},
 {'source': 0, 'target': 6, 'weight': 3},
 {'source': 0, 'target': 8, 'weight': 4},
 {'source': 0, 'target': 13, 'weight': 10},
 {'source': 0, 'target': 14, 'weight': 6},
 {'source': 0, 'target': 19, 'weight': 2},
 {'source': 0, 'target': 17, 'weight': 12},
 {'source': 0, 'target': 10, 'weight': 6},
 {'source': 0, 'target': 18, 'weight': 3},
 {'source': 0, 'target': 20, 'weight': 5},
 {'source': 0, 'target': 22, 'weight': 6},
 {'source': 0, 'target': 23, 'weight': 27},
 {'source': 0, 'target': 24, 'weight': 15},
 {'source': 0, 'target': 25, 'weight': 10},
 {'source': 0, 'target': 30, 'weight': 3},
 {'source': 0, 'target': 31, 'weight': 1},
 {'source': 0, 'target': 34, 'weight': 1},
 {'source': 0, 'target': 35, 'weight': 1},
 {'sourc

In [158]:
doctor_1_file = open("doctor_1.txt", "w+")
doctor_1_file.write("{\"nodes\": ")
doctor_1_file.write(str(nodes_doctor_1).replace("'", "\""))
doctor_1_file.write(", \"links\": ")
doctor_1_file.write(str(links_doctor_1).replace("'", "\""))
doctor_1_file.write("}")
doctor_1_file.close()

In [162]:
# generalize to all doctor groups
def create_node_json(doctor_graph):
    doctor_nodes = []
    for char_name, char_node in doctor_graph.node_names.items():
        doctor_nodes.append({
            "character": char_name,
            "id": char_node.id,
            "influence": len(char_node.adjacents),
        })
    return doctor_nodes

def create_link_json(doctor_graph):
    doctor_links = []
    for char_name, char_node in doctor_graph.node_names.items():
        char_adj = list(char_node.adjacents)
        for adj in char_adj:
            target_node = doctor_graph.node_names[adj]
            doctor_links.append({
                "source": char_node.id,
                "target": target_node.id,
                "weight": char_node.adjacents[target_node.name],
            })
    return doctor_links
    
def create_data_file(doctor_filename, doctor_nodes, doctor_links):
    doctor_file = open(doctor_filename, "w+")
    doctor_file.write("{\"nodes\": ")
    doctor_file.write(str(doctor_nodes).replace("'", "\""))
    doctor_file.write(", \"links\": ")
    doctor_file.write(str(doctor_links).replace("'", "\""))
    doctor_file.write("}")
    doctor_file.close()

In [167]:
start = 13
end = 14

doctor_season_range = {
    "2": range(4, 7),
    "3": range(7, 12),
    "4": range(12, 19),
    "5": range(19, 22),
    "6": range(21, 24),
    "7": range(24, 27),
    "9": range(27, 28),
    "10": range(28, 31),
    "11": range(31, 34),
    "12": range(34, 37),
    "13": range(37, 38)
}

doctor_ep_range = {
    "2": {"4": range(3, 10), "5": range(1, 8), "6": range(1, 8)},
    "3": {"7": range(1, 5), "8": range(1, 6), "9": range(1, 6), "10": range(1, 6), "11": range(1, 6)},
    "4": {"12": range(1, 6), "13": range(1, 7), "14": range(1, 7), "15": range(1, 7), "16": range(1, 7), "17": range(1, 7), "18": range(1, 8)},
    "5": {"19": range(1, 8), "20": range(1, 8), "21": range(1, 7)},
    "6": {"21": range(7, 8), "22": range(1, 7), "23": range(1, 5)},
    "7": {"24": range(1, 5), "25": range(1, 5), "26": range(1, 5)},
    "9": {"27": range(1, 14)},
    "10": {"28": range(0, 14), "29": range(0, 14), "30": range(0, 19)},
    "11": {"31": range(1, 14), "32": range(0, 14), "33": range(0, 17)},
    "12": {"34": range(1, 13), "35": range(0, 14), "36": range(0, 14)},
    "13": {"37": range(1, 12)} 
}

for each_doctor in range(start, end):
    if each_doctor == 8:
        continue
    curr_ep_range = doctor_ep_range[str(each_doctor)]
    curr_season_range = doctor_season_range[str(each_doctor)]
    curr_doctor_graph = doctor_graph(curr_ep_range, curr_season_range)
    doctor_node_json = create_node_json(curr_doctor_graph)
    print(doctor_node_json)
    doctor_link_json = create_link_json(curr_doctor_graph)
    print(doctor_link_json)
    create_data_file("doctors/doctor_" + str(each_doctor) + ".txt", doctor_node_json, doctor_link_json)

1
http://www.chakoteya.net/DoctorWho/37-1.html
111
2
http://www.chakoteya.net/DoctorWho/37-2.html
176
3
http://www.chakoteya.net/DoctorWho/37-3.html
258
4
http://www.chakoteya.net/DoctorWho/37-4.html
351
5
http://www.chakoteya.net/DoctorWho/37-5.html
444
6
http://www.chakoteya.net/DoctorWho/37-6.html
504
7
http://www.chakoteya.net/DoctorWho/37-7.html
581
8
http://www.chakoteya.net/DoctorWho/37-8.html
584
9
http://www.chakoteya.net/DoctorWho/37-9.html
663
10
http://www.chakoteya.net/DoctorWho/37-10.html
754
11
http://www.chakoteya.net/DoctorWho/37-11.html
759
[{'character': 'RAMESH', 'id': 0, 'influence': 18}, {'character': 'SONIA', 'id': 1, 'influence': 18}, {'character': 'RYAN', 'id': 2, 'influence': 90}, {'character': 'GRACE', 'id': 3, 'influence': 32}, {'character': 'JANEY', 'id': 4, 'influence': 18}, {'character': 'KARL', 'id': 5, 'influence': 18}, {'character': 'RAHUL', 'id': 6, 'influence': 18}, {'character': 'DOCRO', 'id': 7, 'influence': 18}, {'character': 'ANNOUNCEMENT', 'id':

In [ ]:
start = 1
end = 14

for each_doctor in range(start, end):
    if each_doctor == 8:
        continue
    curr_ep_range = doctor_ep_range[str(each_doctor)]
    curr_season_range = doctor_season_range[str(each_doctor)]
    curr_doctor_graph = doctor_graph(curr_ep_range, curr_season_range)
    doctor_node_json = create_node_json(curr_doctor_graph)
    print(doctor_node_json)
    doctor_link_json = create_link_json(curr_doctor_graph)
    print(doctor_link_json)
    create_data_file("doctors/doctor_" + str(each_doctor) + ".txt", doctor_node_json, doctor_link_json)